In [40]:
#前処理後のindex作るための処理
import os
import elasticsearch as es
import math
import numpy as np
from itertools import chain
import sys
import tensorflow as tf
import matplotlib.pyplot as plt

ES_HOST = "localhost"
escon = es.Elasticsearch(ES_HOST, port=9200,timeout=300)
GACRP_HOME=os.environ['GACRP_HOME']

# ESのデータ量依存設定値のグローバル変数化
# scrollのタイムアウト時間
es_s_time='60m'

maxPartition=90


In [41]:
#訓練データの期間
 #for化済み
    
#評価データの期間
evalIndataStartYmd=20180101
evalIndataEndYmd=20180631
evalForecastStartYmd=20180801
evalForecastEndYmd=20180931

#結果提出データの期間
submitIndataStartYmd=20180501
submitIndataEndYmd=20181030

#教師データから変換した項目数（one-hot後）
# Todo 配列の形式から自動設定にする。
num_elements=27

In [42]:
def num2date(num,flg):
    mm=num%12
    yy=2016+int(num/12)
    if mm == 0:
        mm=12
        yy-=1
    if flg=="start":
        dd=1
    else:
        dd=31
    return(yy*10000+mm*100+dd)

In [43]:
# 元データ取得

def getInputData(startymd,endymd):
    es_size=10000
    body = {
      "_source": [
        "fullVisitorId","date",
        "channelGrouping","visitNumber","device_isMobile","geoNetwork_country","totals_hits","totals_bounces",
        "totals_newVisits","trafficSource_source","trafficSource_medium","trafficSource_isTrueDirect","totals_transactionRevenue"
      ],
      "query": {
        "bool": {
          "must": [
            {
              "range": {
                "date": {
                  "gt": startymd,
                  "lt": endymd
                }
              }
            }
          ],
          "must_not": [],
          "should": []
        }
      },
      "from": 0,
      "size": 0,
      "sort": [],
      "aggs": {}
    }
  
    res=escon.search(index='gacrp_v2_index',doc_type='gacrp_v2_type',body=body,scroll=es_s_time,size=es_size)
    es_s_id=res['_scroll_id']
    es_s_size=len(res["hits"]["hits"])
    
    return(res,es_s_id,es_s_size)


In [44]:
#fullVisitorIdごとに、指定した期間のrevenueを取得
def getRevenue(startYmd,endYmd,partitionNumber,maxPartition):
    es_size=10000
    body = {
        "size":0,
        "aggs":{
            "agg_fullVisitorId":{
                "terms":{
                    "field":"fullVisitorId.keyword",
                    "include":{
                        "partition":partitionNumber,
                        "num_partitions":maxPartition
                    },
                    "size":10000
                },
                "aggs":{
                    "agg_revenue":{
                        "sum":{"field":"totals_transactionRevenue"}
                    }
                }
            }
        },
        "query":{
            "bool":{
                "must":[
                    {"range": {"date": {
                                "gt": startYmd,"lt": endYmd
                    }}}
                ]
            }
        }
    }
    #print(body)
    res=escon.search(index='gacrp_v2_index',doc_type='gacrp_v2_type',body=body)
    #revenue=math.log(int(res['aggregations']['agg_revenue']['value'])+1)
    #print(revenue)
    return(res['aggregations']['agg_fullVisitorId']['buckets'])


In [45]:
#oneHotベクトル化
def es2oneHot(res,data):
    for i in range(len(res["hits"]["hits"])):
        row=res["hits"]["hits"][i]["_source"]
        rec=[]

        col_date=row['date']
        col_fullVisitorId=row['fullVisitorId']
        
        if row['channelGrouping']=='Display':
            col_channelGrouping=1
        elif row['channelGrouping']=='Social':
            col_channelGrouping=2
        elif row['channelGrouping']=='Orgranic Search':
            col_channelGrouping=3
        elif row['channelGrouping']=='Direct':
            col_channelGrouping=4
        elif row['channelGrouping']=='Referral':
            col_channelGrouping=5
        elif row['channelGrouping']=='Paid Search':
            col_channelGrouping=6
        elif row['channelGrouping']=='Affiliates':
            col_channelGrouping=7
        else:
            col_channelGrouping=0
        oneHot_col_channelGrouping=np.eye(7+1)[col_channelGrouping]
        
        #visitNumberは最大を10で打ち切って0～1にnormalize。
        col_visitNumber=min(int(row['visitNumber']),10)/10.0 

        if row['device_isMobile']=="True":
            col_isMobile=1
        else:
            col_isMobile=0

        if row['geoNetwork_country']=="United States":
            col_country =1
        elif row['geoNetwork_country']=="Canada":
            col_country =2
        elif row['geoNetwork_country']=="Japan":
            col_country=3
        else:
            col_country=0
        oneHot_col_country=np.eye(3+1)[col_country]
    
        col_hits=math.log10(int(row['totals_hits'])*1.0)
    
        if row['totals_bounces']=="1":
            col_bounces=1
        else:
            col_bounces=0

        if row['totals_newVisits']== "1":
            col_newVisits=1
        else:
            col_newVisits=0
    
        if row['trafficSource_source']=="mail.googleplex.com" or row['trafficSource_source']=="dfa":
            col_source =1
        elif row['trafficSource_source']=="(direct)":
            col_source=2
        elif row['trafficSource_source']=="google":
            col_source=3
        else:
            col_source=0
        oneHot_col_source=np.eye(3+1)[col_source]
        
        if row['trafficSource_medium']=="cpm":
            col_medium=1
        elif row['trafficSource_medium']=="organic":
            col_medium=2
        elif row['trafficSource_medium']=="referral":
            col_medium=3
        elif row['trafficSource_medium']=="(none)":
            col_medium=4
        else:
            col_medium=0
        oneHot_col_medium=np.eye(4+1)[col_medium]
    
        if row['trafficSource_isTrueDirect']=="1":
            col_isTrueDirect=1
        else:
            col_isTrueDirect=0
    
        #revenue = math.log(int(row['transactionRevenue'])+1)
    
        l=[oneHot_col_channelGrouping,[col_visitNumber],[col_isMobile],oneHot_col_country,[col_hits],[col_bounces],[col_newVisits],oneHot_col_source,oneHot_col_medium,[col_isTrueDirect]]
        data.append([col_fullVisitorId,list(chain.from_iterable(l))])
        #data.append([col_fullVisitorId,l])
        #ans.append(revenue)
    
    return(data)

In [46]:
#rowdataを対象期間のfullVisitorIdごとに集約する
def transformInputdata(rowdata):
    rowdata=sorted(rowdata,key=lambda x:x[0])
    print("ソート完了")
    key_fullVisitorId=rowdata[0][0]
    sumdata=[]

    tmp_indata=[]
    tmp_indata.append(rowdata[0][1])
    tmp_cnt=1
    for i in range(1,len(rowdata)):
        #if i %100000==0:
        #    print("集約処理実行中："+str(i) )
        if key_fullVisitorId==rowdata[i][0]:
            tmp_indata.append(rowdata[i][1])
            tmp_cnt+=1
        else:
            #列方向に平均をとってappend
            #print(tmp_indata)
            #print(len(rowdata[i][1]))
            tmp_np = np.empty((0,num_elements),float)
            for j in range(tmp_cnt):
                tmp_np = np.append(tmp_np,np.array([tmp_indata[j]]),axis=0)
            sumdata.append([key_fullVisitorId,np.average(tmp_np,axis=0).tolist()])
        #ans.append(getRevenueByFullVisitorId(key_fullVisitorId,forecastStartYmd,forecastEndYmd))
            key_fullVisitorId=rowdata[i][0]
            tmp_indata=[]
            tmp_indata.append(rowdata[i][1])
            tmp_cnt=1

    tmp_np = np.empty((0,num_elements),float)
    for j in range(tmp_cnt):
        tmp_np = np.append(tmp_np,np.array([tmp_indata[j]]),axis=0)
    sumdata.append([key_fullVisitorId,np.average(tmp_np,axis=0)])
    print(len(sumdata))
    
    return(sumdata)

In [47]:
#集約後のfullVisitorIdにRevenueをjoinする
#print(tmpans[0])
def joinRevenue(sumdata,tmpans):
    ans=[]
    #npans=np.array(tmpans)
    #print(npans[0])
    print(sumdata[0][0])

    tmpans=sorted(tmpans,key=lambda x:x[0])
    #print("tmpansソート完了"+str(len(tmpans)) )
    #print(tmpans[0])

    key_ans=tmpans[0][0]
    ix_ans=0
    key_data=sumdata[0][0]
    ix_data=0
    inf = '9999999999999999999'

    while key_ans != inf or key_data != inf:
        if key_ans == key_data:
            ans.append(tmpans[ix_ans][1])
            ix_data+=1
            ix_ans+=1
        elif key_ans > key_data:
            ans.append(0)
            ix_data+=1
        else:
            ix_ans+=1
        if ix_ans >= len(tmpans):
            key_ans=inf
        else:
            key_ans=tmpans[ix_ans][0]

        if ix_data >= len(sumdata):
            key_data=inf
        else:
            key_data=sumdata[ix_data][0]

    print("回答データ読込件数:"+str(len(ans)) )
    print("回答データ合計値　:"+str(sum(ans)) )
    
    return(ans)


In [48]:
#訓練データ読み込み
#rowdata=[]
print("モジュールロード完了")
ans = []
sumdata=[]

#for i in range(8,29+1):
for i in range(8,29+1,2):
    rowdata=[]
    indataStartYmd=num2date(i,"start")
    indataEndYmd  =num2date(i+5,"end")
    print("訓練データ読込開始:"+str(indataStartYmd) )
    res,es_s_id,es_s_size=getInputData(indataStartYmd,indataEndYmd)
    
    while(es_s_size>0):
        rowdata=es2oneHot(res,rowdata)
        res=escon.scroll(scroll_id=es_s_id,scroll=es_s_time)
        es_s_id=res['_scroll_id']
        es_s_size=len(res["hits"]["hits"])

    print("訓練データ読込件数:"+str(len(rowdata)) )
    
    tmpdata=transformInputdata(rowdata)
    for j in range(len(tmpdata)):
        sumdata.append(tmpdata[j])
    
    #partitionでぐるぐる回してtransactionRevenueをとってくる
    #sumdataに合わせてansを当てて設定する。

    tmpans=[]
    forecastStartYmd=num2date(i+7,"start")
    forecastEndYmd=num2date(i+8,"end")
    for partitionNumber in range(maxPartition):
        #print("getRevenue:",forecastStartYmd,forecastEndYmd)
        res=getRevenue(forecastStartYmd,forecastEndYmd,partitionNumber,maxPartition)
        #print("len(res):"+ str(len(res)))
        for i in range(len(res)):
            tmpans.append([res[i]['key'], math.log(int(res[i]['agg_revenue']['value'])+1)])

    tmpans2=joinRevenue(tmpdata,tmpans)

    for j in range(len(tmpans2)):
        ans.append(tmpans2[j])
    



モジュールロード完了
訓練データ読込開始:20160801
訓練データ読込件数:497129
ソート完了
401437
0000010278554503158
訓練回答データ読込件数:401437
訓練回答データ合計値　:3230.959671671152
訓練データ読込開始:20160901
訓練データ読込件数:485817
ソート完了
392638
0000010278554503158
訓練回答データ読込件数:392638
訓練回答データ合計値　:3386.622746142747
訓練データ読込開始:20161001
訓練データ読込件数:483546
ソート完了
391028
0000010278554503158
訓練回答データ読込件数:391028
訓練回答データ合計値　:3011.8800260493176
訓練データ読込開始:20161101
訓練データ読込件数:453288
ソート完了
362956
0000020424342248747
訓練回答データ読込件数:362956
訓練回答データ合計値　:2249.285945453647
訓練データ読込開始:20161201
訓練データ読込件数:401922
ソート完了
314429
0000027376579751715
訓練回答データ読込件数:314429
訓練回答データ合計値　:3402.6668672105106
訓練データ読込開始:20170101
訓練データ読込件数:391500
ソート完了
307043
0000027376579751715
訓練回答データ読込件数:307043
訓練回答データ合計値　:3584.9677121025356
訓練データ読込開始:20170201
訓練データ読込件数:394914
ソート完了
309742
0000027376579751715
訓練回答データ読込件数:309742
訓練回答データ合計値　:2126.7224185005557
訓練データ読込開始:20170301
訓練データ読込件数:417239
ソート完了
326817
0000039460501403861
訓練回答データ読込件数:326817
訓練回答データ合計値　:1914.1731312699267
訓練データ読込開始:20170401
訓練データ読込件数:437759
ソート完

In [49]:
#評価データ読み込み
evalrowdata=[]
evalans = []

# 評価データは期間を固定

res,es_s_id,es_s_size=getInputData(evalIndataStartYmd,evalIndataEndYmd)
while(es_s_size>0):
    rowdata=es2oneHot(res,evalrowdata)
    res=escon.scroll(scroll_id=es_s_id,scroll=es_s_time)
    es_s_id=res['_scroll_id']
    es_s_size=len(res["hits"]["hits"])

print("評価データ読込完了:"+str(len(evalrowdata)) )

evalsumdata=transformInputdata(evalrowdata)

#partitionでぐるぐる回してtransactionRevenueをとってくる
#sumdataに合わせてansを当てて設定する。

evaltmpans=[]
for partitionNumber in range(maxPartition):
    #print(partitionNumber)
    res=getRevenue(evalForecastStartYmd,evalForecastEndYmd,partitionNumber,maxPartition)
    for i in range(len(res)):
        evaltmpans.append([res[i]['key'], math.log(int(res[i]['agg_revenue']['value'])+1)])

evalans=joinRevenue(evalsumdata,evaltmpans)


評価データ読込完了:502121
ソート完了
386733
0000039738481224681
訓練回答データ読込件数:386733
訓練回答データ合計値　:2788.7000372286234


In [62]:
#tensorflowセットアップ
datalength=len(sumdata)

#print(data[0],ans[0])
#inputData=tf.transpose(data)
debugFlg=False
#batch_size=int(datalength/2)
#batch_size=datalength
batch_size=len(evalans)

P=int(num_elements*3)
Q=int(num_elements*3)

num_rows=len(sumdata)

##訓練データをtensorflowが食えるように変換
data=np.empty((0,num_elements))
tmp_array=data.tolist()
for i in range(len(sumdata)):
    tmp_array.append(np.array(sumdata[i][1]))
data=np.asarray(tmp_array)        
print(data.shape)

inputData=data
inputAns=np.reshape(ans,(num_rows,1))


##評価データを同様に設定
evalnum_rows=len(evalsumdata)
evaldata=np.empty((0,num_elements))
evaltmp_array=evaldata.tolist()
for i in range(len(evalsumdata)):
    evaltmp_array.append(np.array(evalsumdata[i][1]))
evaldata=np.asarray(evaltmp_array)
print(evaldata.shape)

evalInputData=evaldata
evalAns=np.reshape(evalans,(evalnum_rows,1))


#inputData=np.reshape(np.delete(sumdata,0,1),(num_rows,num_elements))
#inputData=np.reshape(sumdata[:][1],(num_rows,num_elements))
##inputData=np.reshape(data,(num_rows,num_elements))
#inputData=data,(num_rows,num_elements)
x =tf.placeholder("float",[None,num_elements])
y_=tf.placeholder("float",[None,1])

a=tf.Variable( tf.random_uniform([num_elements,1],-1.0,1.0),name="weights")    
w1=tf.Variable( tf.random_uniform([num_elements,P],-1.0,1.0 )  )
w2=tf.Variable( tf.random_uniform([P,Q],-1.0,1.0 )  )
w3=tf.Variable( tf.random_uniform([Q,1],-1.0,1.0 )  )

#a=tf.Variable( tf.random_uniform([[18]],-1.0,1.0),name="weights")    
#b=tf.Variable(tf.zeros([1,1]),name="intercept")
b=tf.Variable(tf.random_normal([P]),name="intercept")
print("x:",x.shape,"a:",a.shape)
#y=tf.matmul(tf.transpose(a),x)+b
#y=tf.matmul(x,a)

x2=tf.sigmoid(tf.matmul(x,w1))+b
#y=tf.matmul(x2,w2)
x3=tf.sigmoid(tf.matmul(x2,w2))
y =tf.matmul(x3,w3)

init = tf.global_variables_initializer()
#loss = tf.reduce_sum(tf.square(y_-y))/batch_size
loss = tf.reduce_mean(tf.square(y_-y))
forecastRevenue = tf.reduce_sum(tf.square(y))
train_step=tf.train.GradientDescentOptimizer(0.01).minimize(loss)


(4309411, 27)
(386733, 27)
x: (?, 27) a: (27, 1)


In [63]:
del data,evaldata,tmp_array,evaltmp_array

outpath=GACRP_HOME+"/result.dat"

In [69]:
#データ表示用セットアップ
# 基準となる全部ゼロ時の値(訓練データ)
tmp=0
for i in range(len(ans)):
    tmp+=ans[i]*ans[i]/len(ans)
print("sum of math.log(transactionRevenue) train:"+str(tmp) )
# 基準となる全部ゼロ時の値(評価データ)
tmp=0
for i in range(len(evalans)):
    tmp+=evalans[i]*evalans[i]/len(evalans)
print("sum of math.log(transactionRevenue) eval :"+str(tmp) )

graphLoss=[]
graphEval=[]
graphStep=[]
graphNum=30
repeatAll=30
debugFlg=False
with tf.Session() as sess:
    sess.run(init)
    print('初期状態')
    inpShape=inputData[0:batch_size]
    ansShape=inputAns[0:batch_size]
    #print('誤差' + str(sess.run(loss, feed_dict={x: inputData[0:batch_size], y_: inputAns[0:batch_size]})))
    ##print('誤差' + str(sess.run(loss, feed_dict={x: inpShape, y_: ansShape})))
    #print("weigths: %f, intercept: %f" % (sess.run(a), sess.run(b) ))
    #print("input:",inputAns[batch_size*step+1:batch_size*step+1+batch_size])
    ##print("weigths:",sess.run(a),"intercept:",sess.run(b) )
    
    for step in range(repeatAll):
        if (batch_size*step) %datalength < (batch_size*step+batch_size) %datalength :
            batchInputData= inputData[(batch_size*step)%datalength:(batch_size*step+batch_size)%datalength]
            batchInputAns = inputAns[(batch_size*step)%datalength:(batch_size*step+batch_size)%datalength]
        else:
            batchInputData= inputData[(batch_size*step)%datalength:min( (batch_size*step+batch_size)%datalength,datalength-1 )]
            batchInputAns = inputAns[(batch_size*step)%datalength:min( (batch_size*step+batch_size)%datalength,datalength-1) ]
        sess.run(train_step, feed_dict={x: batchInputData, y_: batchInputAns })
        if (step+1) % (repeatAll/graphNum) == 0:
            print('\nStep: %s' % (step+1))
            print('誤差' + str(sess.run(loss, feed_dict={x: batchInputData, y_: batchInputAns })))
            print('評価' + str(sess.run(loss, feed_dict={x: evalInputData, y_: evalAns })))
            print('評価時予想額' + str(sess.run(forecastRevenue, feed_dict={x: evalInputData, y_: evalAns })))
            if debugFlg==True:
                print("weights:",sess.run(w1), " intercept:", sess.run(b) )
            graphLoss.append( sess.run(loss, feed_dict={x: batchInputData, y_: batchInputAns }) )
            graphEval.append( sess.run(loss, feed_dict={x: evalInputData, y_: evalAns }))
            graphStep.append(step)
    #最終状態を表示
    print("weights1:",sess.run(w1),"weights2:",sess.run(w2), " intercept:", sess.run(b))
    #予想結果
    #with open(outpath, mode='w') as f:
    #   f.write( str(sess.run(y, feed_dict={x: evalInputData, y_: evalAns }))  )
    np.savetxt(outpath,sess.run(y, feed_dict={x: evalInputData, y_: evalAns }), delimiter=',')

sum of math.log(transactionRevenue) train:0.13626994362548103
sum of math.log(transactionRevenue) eval :0.13017805686484843
初期状態

Step: 1
誤差0.38398945
評価0.62229514
評価時予想額184207.19

Step: 2
誤差0.361084
評価0.58989114
評価時予想額172839.47

Step: 3
誤差0.35267714
評価0.58107764
評価時予想額168559.77

Step: 4
誤差0.33827847
評価0.56564283
評価時予想額162587.89

Step: 5
誤差0.45699435
評価0.61199963
評価時予想額179611.05

Step: 6
誤差0.42908055
評価0.59350014
評価時予想額172503.19

Step: 7
誤差0.3684515
評価0.611071
評価時予想額179016.78

Step: 8
誤差0.33020976
評価0.58516824
評価時予想額169123.92

Step: 9
誤差0.33735976
評価0.58163494
評価時予想額167698.64

Step: 10
誤差0.3144475
評価0.5540365
評価時予想額157177.92

Step: 11
誤差0.30471355
評価0.53786445
評価時予想額151004.81

Step: 12
誤差nan
評価0.53786445
評価時予想額151004.81

Step: 13
誤差0.2806197
評価0.4235087
評価時予想額108908.086

Step: 14
誤差0.271919
評価0.43731815
評価時予想額113133.305

Step: 15
誤差0.25590426
評価0.4194514
評価時予想額106438.05

Step: 16
誤差0.3348452
評価0.4487896
評価時予想額117204.3

Step: 17
誤差0.3440977
評価0.4477863
評価時予想額116752.78

Step: 18
誤差0.2691

In [ ]:
plt.plot(graphStep,graphLoss,graphStep,graphEval)

In [ ]:
## todo 学習した結果の重みを使って予想を計算する。
## sess.runした結果を整形すればいいはず。

In [ ]:
import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 1000000: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))
        